In [1]:
using Catan
using CatanLearning
using Random
using CSV
using MLJ
using DataFrames
import DataFramesMeta as DFM
using DelimitedFiles
using MLJDecisionTreeInterface
using DecisionTree
import MLJModelInterface
const MMI = MLJModelInterface

┌ Warning: attempting to remove probably stale pidfile
│   path = /home/bryan/.julia/compiled/v1.10/CatanLearning/c4wiD_P1pe2.ji.pidfile
└ @ FileWatching.Pidfile /home/bryan/.julia/juliaup/julia-1.10.4+0.x64.linux.gnu/share/julia/stdlib/v1.10/FileWatching/src/pidfile.jl:244


DIR: /home/bryan/Projects/Catan/CatanLearning.jl/src


┌ Info: Registering EmpathRobotPlayer
└ @ Catan /home/bryan/Projects/Catan/CatanEngine.jl/src/players/structs.jl:120


MLJModelInterface

In [2]:
function naive_model_proba(X)
    unscaled_feature = X[:CountSettlement] + X[:CountCity] + 2*(X[:HasLargestArmy] + X[:HasLongestRoad])
    return min(unscaled_feature / 10, 1.0)
end

MMI.@mlj_model mutable struct NaivePublicVictoryPoints <: MLJModelInterface.Deterministic
end

MMI.fit(::NaivePublicVictoryPoints, verbosity, data...) = (nothing, nothing, nothing)
MMI.predict(model::NaivePublicVictoryPoints, fitresult, new_data...) = naive_model_proba(new_data)

model = NaivePublicVictoryPoints()
naive_machine = machine(model, nothing, nothing) |> MLJ.fit!

┌ Info: Training machine(NaivePublicVictoryPoints(), …).
└ @ MLJBase /home/bryan/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


trained Machine; caches model-specific representations of data
  model: NaivePublicVictoryPoints()
  args: 
    1:	Source @323 ⏎ Nothing
    2:	Source @676 ⏎ Nothing


In [3]:
config_file = "feature_generation.toml"
(configs, player_configs, logger) = Catan.parse_configs(config_file)
rf_machine = CatanLearning.try_load_model_from_csv(player_configs)
global player_configs

Savefile set to savefile.txt
Configs loaded from feature_generation.toml


In [4]:

(configs, player_configs, logger) = Catan.parse_configs(config_file)

tree = Base.invokelatest(@load RandomForestClassifier pkg=DecisionTree verbosity=0)
public_mach = CatanLearning.train_public_model_from_csv(tree, player_configs)



Savefile set to savefile.txt
Configs loaded from feature_generation.toml


Evaluating over 100 metamodels: 100%[=========================] Time: 0:05:03


trained Machine; caches model-specific representations of data
  model: RandomForestClassifier(max_depth = -1, …)
  args: 
    1:	Source @253 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}
    2:	Source @009 ⏎ AbstractVector{OrderedFactor{2}}


In [10]:
MLJ.save(player_configs["PUBLIC_MODEL"], public_mach)

In [6]:
CatanLearning.feature_library

Dict{Symbol, CatanLearning.Feature} with 41 entries:
  :SumStoneDiceWeight            => Feature(:SumStoneDiceWeight, Int8, 4.0, 186…
  :CountDevCardsMonopoly         => Feature(:CountDevCardsMonopoly, Int8, 0.0, …
  :HasLargestArmy                => Feature(:HasLargestArmy, Bool, 0.0, 1.0)
  :CountTotalWood                => Feature(:CountTotalWood, Int8, 0.0, 20.0)
  :CountDevCardsUsedKnight       => Feature(:CountDevCardsUsedKnight, Int8, 0.0…
  :SumGrainDiceWeight            => Feature(:SumGrainDiceWeight, Int8, 4.0, 186…
  :PortStone                     => Feature(:PortStone, Int8, 0.0, 1.0)
  :CountHandBrick                => Feature(:CountHandBrick, Int8, 0.0, 20.0)
  :SumBrickDiceWeight            => Feature(:SumBrickDiceWeight, Int8, 4.0, 186…
  :HasLongestRoad                => Feature(:HasLongestRoad, Bool, 0.0, 1.0)
  :CountHandPasture              => Feature(:CountHandPasture, Int8, 0.0, 20.0)
  :CountDevCardsYearOfPlenty     => Feature(:CountDevCardsYearOfPlenty, Int8, 0…

In [7]:
CatanLearning.coerce_feature_types!(df)
schema(df)

UndefVarError: UndefVarError: `df` not defined

In [8]:
X

UndefVarError: UndefVarError: `X` not defined